In [1]:
%matplotlib inline
""" handling files support packages """
from glob import glob

""" logic support packages """
import numpy as np
import pandas as pd

""" plot support packages """
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from IPython.display import Image
import pydotplus

""" image trasformation packages """
from PIL import Image
import skimage.io as skio

""" statistical data visualization packages"""
import seaborn as sns

""" machine learning functions """
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.externals import joblib

""" seaborn configurations """
sns.set_style('white')
sns.set_context('talk')
plt.rcParams['figure.figsize'] = 20, 10

# Tentativa de com base equilibrada p%

In [2]:
combinacoes30 = pd.read_csv("../combinacoes50_2.txt", header = 0)

In [3]:
le = preprocessing.LabelEncoder()
nomes = []
for i in sorted(glob('../letras/caracter*.png')):
    nomes.append(i.split("/")[1] + "/" + i.split("/")[2])

In [4]:
le.fit(nomes)

LabelEncoder()

In [5]:
combinacoes30["char1"] = le.transform(combinacoes30["char1"])
combinacoes30["char2"] = le.transform(combinacoes30["char2"])

In [6]:
#X_train, X_test, y_train, y_test = train_test_split(combinacoes30[["char1", "char2", "MSE", "ISS", "MSE_centro", "ISS_centro"]],
#                                                                    combinacoes30["resposta"], test_size=0.3, random_state=0)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(combinacoes30[["MSE", "ISS", "MSE_centro", "ISS_centro",
                                                                  "MSE_canny","ISS_canny","MSE_canny_centro","ISS_canny_centro",
                                                                  "MSE_skeleton","ISS_skeleton","MSE_skeleton_centro","ISS_skeleton_centro"]]],
                                                                    combinacoes30["resposta"], test_size=0.3, random_state=0)

## SVM

In [8]:
%%time
clf = SVC(kernel = 'poly', degree = 2, max_iter = 100000)
clf.fit(X_train, y_train)
print(clf.score(X_test, y_test))

0.611632270169
CPU times: user 4.14 s, sys: 8 ms, total: 4.15 s
Wall time: 4.16 s


## Decision Tree

In [9]:
%%time
from sklearn import tree

clf1 = tree.DecisionTreeClassifier(criterion = "gini", min_samples_split = 80, min_samples_leaf = 3)

clf1.fit( X = X_train, y = y_train )
print( clf1.score(X_test, y_test) )

0.619512195122
CPU times: user 36 ms, sys: 0 ns, total: 36 ms
Wall time: 41.9 ms


In [10]:
# Tentativa de plotar a árvore do modelo acima
#dot_data = tree.export_graphviz(clf1, out_file=None, 
#                         feature_names = ["MSE","ISS",
#                                          "MSE_centro","ISS_centro"],  
#                         class_names = combinacoes30["resposta"],  
#                         filled = True, rounded = True,  
#                         special_characters=True)

#graph = pydotplus.graph_from_dot_data(dot_data)  
#Image(graph.create_png())
##graph.write_pdf("perfil_anovalor.pdf")

# Random Forest

In [11]:
from sklearn.ensemble import RandomForestClassifier
clfRF_balanced = RandomForestClassifier()

In [12]:
%%time
clfRF_balanced.fit( X = X_train, y = y_train )
print( clfRF_balanced.score(X_test, y_test) )

0.613133208255
CPU times: user 148 ms, sys: 4 ms, total: 152 ms
Wall time: 149 ms


# Binarizando o melhor modelo

In [13]:
#joblib.dump(clf, 'classifier_balanced.pkl') 

['classifier_balanced.pkl']

# Tentativa com base inteira

In [14]:
combinacoes = pd.read_csv("../combinacoes_2.txt", header = 0)

In [15]:
combinacoes["char1"] = le.transform(combinacoes["char1"])
combinacoes["char2"] = le.transform(combinacoes["char2"])

In [17]:
X_train, X_test, y_train, y_test = train_test_split(combinacoes[["MSE", "ISS", "MSE_centro", "ISS_centro",
                                                                  "MSE_canny","ISS_canny","MSE_canny_centro","ISS_canny_centro",
                                                                  "MSE_skeleton","ISS_skeleton","MSE_skeleton_centro","ISS_skeleton_centro"]]],
                                                                    combinacoes30["resposta"], test_size=0.3, random_state=0)

## SVM

In [18]:
#%time
#clf.fit(X_train, y_train)
#clf.score(X_test, y_test)

# Decision Tree

In [19]:
%%time
clf1.fit( X = X_train, y = y_train )
print( clf1.score(X_test, y_test) )

0.981970257703
CPU times: user 2.02 s, sys: 8 ms, total: 2.03 s
Wall time: 2.04 s


In [20]:
# Tentativa de plotar a árvore do modelo acima
#dot_data = tree.export_graphviz(clf1, out_file=None, 
#                         feature_names = ["char1", "char2","MSE","ISS","MSE_centro","ISS_centro"],  
#                         class_names = combinacoes["resposta"],  
#                         filled = True, rounded = True,  
#                         special_characters=True)

#graph = pydotplus.graph_from_dot_data(dot_data)  
#Image(graph.create_png())
##graph.write_pdf("perfil_anovalor.pdf")

## Random Forest

In [21]:
from sklearn.ensemble import RandomForestClassifier
clfRF = RandomForestClassifier()

In [22]:
%%time
clfRF.fit( X = X_train, y = y_train )
print( clfRF.score(X_test, y_test) )

0.982363387423
CPU times: user 7.99 s, sys: 0 ns, total: 7.99 s
Wall time: 7.99 s


## Binarizando o melhor modelo

In [23]:
#joblib.dump(clf1, 'classifier.pkl') 

['classifier.pkl']

In [24]:
#joblib.dump(clfRF, 'classifierRF.pkl') 

['classifierRF.pkl']

## testes temporários

In [25]:
# alguns testes temporários
from funcoes import ler_letras, compare_images
_, letters_dict = ler_letras("../letras.csv")

img1 = skio.imread("../" + 'letras/caracter0.png')
img2 = skio.imread("../" + 'letras/caracter1.png')

m, s = compare_images(img1, img2)
mc, cs = compare_images(img1[10:40,], img2[10:40,])
    
print(clf1.predict([[m, s, mc, cs]])[0] == 1)

False
